In [5]:
# Level-32 — Real-Time Incremental Learning (fully hardened)
# Fixes: 1-D broadcasting, Series truth ambiguity, CAGR date calc (DatetimeIndex vs RangeIndex)

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta, timezone
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler

# ---------------------------- Config ----------------------------
TICKER       = "AAPL"
YEARS        = 3.0
FREQ         = "1D"
VOL_SPAN     = 50
H_BARS       = 10
CUSUM_GRID   = [0.003, 0.004, 0.006, 0.008, 0.010]
UP_M, DN_M   = 3.0, 3.0
MIN_EVENTS   = 400
RNG_SEED     = 42
ONLINE_BATCH = 50
TC_BP        = 5

np.random.seed(RNG_SEED)

# ---------------------------- Helpers ----------------------------
def utc_now_date():
    return datetime.now(timezone.utc).date()

def load_prices(ticker, years, freq="1D"):
    end = utc_now_date()
    start = (datetime.now(timezone.utc) - timedelta(days=int(365*years + 20))).date()
    df = yf.download(ticker, start=start, end=end, auto_adjust=True, progress=False)
    if df.empty:
        raise SystemExit("No data downloaded.")
    s = df["Adj Close"] if "Adj Close" in df.columns else df["Close"]
    s = s.asfreq("B").ffill()
    s.name = "Close"
    return s

def ewma_vol(r, span=50):
    return r.ewm(span=span, adjust=False).std()

def to_1d_series(r, index=None):
    if isinstance(r, pd.Series):
        return r.astype(float).fillna(0.0)
    if isinstance(r, pd.DataFrame):
        r = r.iloc[:, 0]
    arr = np.ravel(np.asarray(r, dtype=float))
    if index is None:
        index = pd.RangeIndex(len(arr))
    return pd.Series(arr, index=index, dtype=float).fillna(0.0)

def cusum_filter(r, threshold, index=None):
    s = to_1d_series(r, index=index)
    idx, vals = s.index, s.to_numpy()
    s_pos = s_neg = 0.0
    t_events = []
    for i, x in enumerate(vals):
        s_pos = max(0.0, s_pos + x)
        s_neg = min(0.0, s_neg + x)
        if s_pos > threshold:
            s_pos = 0.0
            t_events.append(idx[i])
        elif s_neg < -threshold:
            s_neg = 0.0
            t_events.append(idx[i])
    return pd.DatetimeIndex(t_events).unique().sort_values()

def get_vertical_barriers(t_events, h, index):
    if len(t_events) == 0:
        return pd.Series(dtype="datetime64[ns]")
    out = {}
    for t0 in t_events:
        pos = index.get_indexer([t0])[0]
        t1_pos = min(pos + h, len(index) - 1)
        out[t0] = index[t1_pos]
    return pd.Series(out)

def get_triple_barrier_labels(close, t_events, vbar, up_m, dn_m, daily_vol):
    trgt = daily_vol.reindex(t_events).fillna(method="bfill").fillna(method="ffill")
    df = pd.DataFrame({"t1": vbar.reindex(t_events), "trgt": trgt}, index=t_events).dropna()
    labels = []
    for t0, row in df.iterrows():
        t1 = row["t1"]
        try:
            c0 = float(close.loc[t0].iloc[0]) if isinstance(close.loc[t0], pd.Series) else float(close.loc[t0])
        except Exception:
            continue
        up_lvl = c0 * (1 + up_m * float(row["trgt"]))
        dn_lvl = c0 * (1 - dn_m * float(row["trgt"]))
        segment = close.loc[t0:t1]
        if isinstance(segment, pd.DataFrame):
            segment = segment.iloc[:, 0]
        path = pd.Series(segment.astype(float), index=segment.index)
        if path.empty:
            continue
        path_up = path >= up_lvl
        path_dn = path <= dn_lvl
        hit_up = path_up.idxmax() if path_up.to_numpy().any() else None
        hit_dn = path_dn.idxmax() if path_dn.to_numpy().any() else None
        if (hit_up is not None) and (hit_dn is not None):
            lbl = 1 if hit_up <= hit_dn else 0
            t_end = hit_up if lbl == 1 else hit_dn
        elif hit_up is not None:
            lbl, t_end = 1, hit_up
        elif hit_dn is not None:
            lbl, t_end = 0, hit_dn
        else:
            c1 = float(path.iloc[-1])
            lbl, t_end = (1 if c1 > c0 else 0), t1
        labels.append((t0, t_end, lbl, float(row["trgt"])))
    if not labels:
        return pd.DataFrame(columns=["t1", "label", "trgt"])
    return pd.DataFrame(labels, columns=["t0", "t1", "label", "trgt"]).set_index("t0")

def make_features(close):
    r = close.pct_change().fillna(0.0)
    f = pd.DataFrame(index=close.index)
    f["r1"] = r
    f["r5"] = close.pct_change(5)
    f["r10"] = close.pct_change(10)
    f["mom5"] = close / close.shift(5) - 1
    f["mom10"] = close / close.shift(10) - 1
    f["vol10"] = r.rolling(10).std()
    f["vol20"] = r.rolling(20).std()
    f["z20"] = (close - close.rolling(20).mean()) / (1e-12 + close.rolling(20).std())
    delta = close.diff()
    up = delta.clip(lower=0).rolling(14).mean()
    dn = (-delta.clip(upper=0)).rolling(14).mean()
    rs = up / (1e-12 + dn)
    f["rsi14"] = 100 - 100 / (1 + rs)
    return f.fillna(0.0)

def adaptive_events_and_labels(close, rets, base_H, cusum_grid, vol_span, up_m, dn_m, min_events):
    daily_vol = ewma_vol(to_1d_series(rets, index=close.index), span=vol_span).clip(lower=1e-8)
    used_thr = None
    events_idx = pd.DatetimeIndex([])
    for thr in cusum_grid:
        ev = cusum_filter(rets, thr, index=close.index)
        if len(ev) >= min_events:
            used_thr, events_idx = thr, ev
            break
    if used_thr is None:
        used_thr = min(cusum_grid)
        events_idx = cusum_filter(rets, used_thr, index=close.index)
    vbar = get_vertical_barriers(events_idx, base_H, close.index)
    labels = get_triple_barrier_labels(close, events_idx, vbar, up_m, dn_m, daily_vol).dropna()
    if labels.empty:
        raise SystemExit("No labeled events. Adjust thresholds.")
    print(f"[Adaptive] events={len(labels)}  H={base_H} thr≈{used_thr}")
    return labels, daily_vol, events_idx, base_H, float(used_thr)

def sharpe_ratio(x):
    s = x.std()
    return 0.0 if s == 0 or np.isnan(s) else float(np.sqrt(252) * x.mean() / s)

def drawdown(x):
    cum = (1 + x).cumprod()
    return float((cum / cum.cummax() - 1).min())

# ---------------------------- Main ----------------------------
if __name__ == "__main__":
    close = load_prices(TICKER, YEARS, FREQ)
    rets  = close.pct_change().fillna(0.0)

    # 1) Events & Labels
    labels, _, _, _, _ = adaptive_events_and_labels(
        close, rets, H_BARS, CUSUM_GRID, VOL_SPAN, UP_M, DN_M, MIN_EVENTS
    )

    # 2) Features aligned
    feats = make_features(close)
    X = feats.reindex(labels.index).dropna()
    y = labels.loc[X.index, "label"].astype(int)

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    model = SGDClassifier(loss="log_loss", penalty="l2",
                          learning_rate="optimal", random_state=RNG_SEED)

    proba_all = np.zeros(len(X), dtype=float)
    signal    = np.zeros(len(X), dtype=float)

    # strictly 1-D forward return, aligned to X.index
    ret_next = close.pct_change().shift(-1).reindex(X.index).fillna(0.0)
    if isinstance(ret_next, pd.DataFrame):
        ret_next = ret_next.iloc[:, 0]
    ret_next = ret_next.astype(float).to_numpy().ravel()

    # online/rolling fit
    for i in range(ONLINE_BATCH, len(X)):
        Xb, yb = X_scaled[i - ONLINE_BATCH:i], y.iloc[i - ONLINE_BATCH:i]
        if i == ONLINE_BATCH:
            model.partial_fit(Xb, yb, classes=np.array([0, 1]))
        else:
            model.partial_fit(Xb, yb)
        p = model.predict_proba(X_scaled[i:i + 1])[0, 1]
        proba_all[i] = p
        signal[i]    = 1.0 if p >= 0.55 else 0.0

    # PnL with TC
    pnl_gross = (signal * ret_next).ravel()
    toggle    = np.abs(np.diff(signal, prepend=0.0))
    tc        = toggle * (TC_BP / 1e4)
    pnl_net   = pnl_gross - tc

    # 🔧 Index PnL by event dates to keep a DatetimeIndex for CAGR;
    # if not DatetimeIndex, we fall back to N/252 years.
    pnl_series = pd.Series(np.ravel(pnl_net), index=X.index)

    sharpe = sharpe_ratio(pnl_series)
    cum    = (1.0 + pnl_series).cumprod()

    if isinstance(cum.index, pd.DatetimeIndex):
        yrs = (cum.index[-1] - cum.index[0]).days / 365.25
    else:
        yrs = len(cum) / 252.0  # fallback

    cagr = float(cum.iloc[-1] ** (1 / max(1e-9, yrs)) - 1.0)
    dd   = drawdown(pnl_series)
    hit  = float(((signal > 0) & (ret_next > 0)).sum() / max(1, (signal > 0).sum()))

    print("\n=== Real-Time Incremental Backtest (Level-32) ===")
    print(f"Sharpe: {sharpe:.3f}  CAGR: {100*cagr:.2f}%  MaxDD: {100*dd:.2f}%  Hit%: {100*hit:.2f}%")
    print(f"Avg turnover: {float(toggle.mean()):.3f}  Avg TC (bp): {float((tc * 1e4).mean()):.2f}")

    out = pd.DataFrame(
        {"proba": proba_all, "signal": signal, "ret_next": ret_next, "pnl_net": pnl_net},
        index=X.index
    )
    name = f"{TICKER}_level32_online_backtest.csv"
    out.to_csv(name)
    print(f"\nSaved: {name}")


[Adaptive] events=627  H=10 thr≈0.003

=== Real-Time Incremental Backtest (Level-32) ===
Sharpe: 1.272  CAGR: 20.50%  MaxDD: -18.85%  Hit%: 55.05%
Avg turnover: 0.198  Avg TC (bp): 0.99

Saved: AAPL_level32_online_backtest.csv
